## Bitcoin data set

In [68]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime

In [69]:
data_bt_usd = pd.read_csv("csv files/BTC_USD.csv")
data_bt_usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   1827 non-null   object
 1   Şimdi   1827 non-null   object
 2   Açılış  1827 non-null   object
 3   Yüksek  1827 non-null   object
 4   Düşük   1827 non-null   object
 5   Hac.    1827 non-null   object
 6   Fark %  1827 non-null   object
dtypes: object(7)
memory usage: 100.0+ KB


In [70]:
df = data_bt_usd.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)
df

,Tarih,Şimdi
0,01.01.2023,"16.593,0"
1,31.12.2022,"16.564,0"
2,30.12.2022,"16.618,0"
3,29.12.2022,"16.658,0"
4,28.12.2022,"16.558,0"
...,...,...
1822,05.01.2018,"16.917,0"
1823,04.01.2018,"15.160,0"
1824,03.01.2018,"15.155,0"
1825,02.01.2018,"14.709,8"


In [71]:
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', ''))/10000)

## LSTM

In [72]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])




[0.0016593 0.0016564 0.0016618 0.0016658 0.0016558 0.0016719 0.0016946] 0.0016847000000000001
[0.0016564 0.0016618 0.0016658 0.0016558 0.0016719 0.0016946 0.0016847] 0.0016864
[0.0016618 0.0016658 0.0016558 0.0016719 0.0016946 0.0016847 0.0016864] 0.0016802
[0.0016658 0.0016558 0.0016719 0.0016946 0.0016847 0.0016864 0.0016802] 0.0016838
[0.0016558 0.0016719 0.0016946 0.0016847 0.0016864 0.0016802 0.0016838] 0.0016844999999999998
[0.0016719 0.0016946 0.0016847 0.0016864 0.0016802 0.0016838 0.0016845] 0.0016925
[0.0016946 0.0016847 0.0016864 0.0016802 0.0016838 0.0016845 0.0016925] 0.0016448
[0.0016847 0.0016864 0.0016802 0.0016838 0.0016845 0.0016925 0.0016448] 0.0016757000000000002
[0.0016864 0.0016802 0.0016838 0.0016845 0.0016925 0.0016448 0.0016757] 0.0016792999999999999
[0.0016802 0.0016838 0.0016845 0.0016925 0.0016448 0.0016757 0.0016793] 0.0016642
[0.0016838 0.0016845 0.0016925 0.0016448 0.0016757 0.0016793 0.0016642] 0.0017372
[0.0016845 0.0016925 0.0016448 0.0016757 0.0016793

In [73]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
57/57 [==============================] - 3s 23ms/step - loss: 2.9782e-06
Epoch 2/200
57/57 [==============================] - 2s 27ms/step - loss: 1.3681e-06
Epoch 3/200
57/57 [==============================] - 1s 18ms/step - loss: 9.1233e-07
Epoch 4/200
57/57 [==============================] - 2s 27ms/step - loss: 2.3277e-07
Epoch 5/200
57/57 [==============================] - 1s 21ms/step - loss: 7.6420e-08
Epoch 6/200
57/57 [==============================] - 1s 22ms/step - loss: 6.8934e-08
Epoch 7/200
57/57 [==============================] - 1s 19ms/step - loss: 3.3974e-08
Epoch 8/200
57/57 [==============================] - 1s 18ms/step - loss: 6.3321e-08
Epoch 9/200
57/57 [==============================] - 1s 18ms/step - loss: 1.0712e-07
Epoch 10/200
57/57 [==============================] - 1s 18ms/step - loss: 1.4981e-07
Epoch 11/200
57/57 [==============================] - 1s 18ms/step - loss: 1.4302e-07
Epoch 12/200
57/57 [==============================] - 1s 25ms/s

In [74]:
# demonstrate prediction
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],df['Şimdi'][3], df['Şimdi'][4], df['Şimdi'][5], df['Şimdi'][6]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

## Daily 

In [75]:
x = datetime.datetime.now()
print("Prediction:",round(yhat[0][0]*10000), "$ Bitcoin/USD in" ,x.strftime('%x'))

Prediction: 15 $ Bitcoin/USD in 01/01/23


## Weekly

In [76]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],df['Şimdi'][3], df['Şimdi'][4], df['Şimdi'][5], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], df['Şimdi'][3], df['Şimdi'][4], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], df['Şimdi'][3],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2] ,float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],float(yhat_5), float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_6 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_6), float(yhat_5),float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_7 = model.predict(x_input, verbose=0)

x_input = array([float(yhat_7), float(yhat_6), float(yhat_5),float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_8 = model.predict(x_input, verbose=0)


yhat   = float(yhat)*10000
yhat_2 = float(yhat_2)*10000
yhat_3 = float(yhat_3)*10000
yhat_4 = float(yhat_4)*10000
yhat_5 = float(yhat_5)*10000
yhat_6 = float(yhat_6)*10000
yhat_7 = float(yhat_7)*10000
yhat_8 = float(yhat_8)*10000


In [77]:
print(f"Monday: {round(yhat,3)} Tuesday : {round(yhat_2,3)} Wednesday: {round(yhat_3,3)} Thursday: {round(yhat_4,3)} Friday: {round(yhat_5,3)}  $ Bitcoin/USD ")


Monday: 15.349 Tuesday : 14.63 Wednesday: 14.093 Thursday: 13.737 Friday: 13.507  $ Bitcoin/USD 


In [78]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Week -> Prediction: {round(yhat_8,2)} $ Bitcoin/USD ")


Next Week -> Prediction: 13.29 $ Bitcoin/USD 


## Monthly

In [79]:
import array
exchange = array.array('f',[])

for i in range(len(df)):
    if i % 7 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])

In [80]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[0.0016593 0.0016847 0.0016757 0.0017109 0.0017128 0.0016432 0.0016281] 0.0016344999894499779
[0.0016847 0.0016757 0.0017109 0.0017128 0.0016432 0.0016281 0.0016345] 0.0020932001061737537
[0.0016757 0.0017109 0.0017128 0.0016432 0.0016281 0.0016345 0.0020932] 0.0020647000055760145
[0.0017109 0.0017128 0.0016432 0.0016281 0.0016345 0.0020932 0.0020647] 0.0019584998954087496
[0.0017128 0.0016432 0.0016281 0.0016345 0.0020932 0.0020647 0.0019585] 0.0019269000040367246
[0.0016432 0.0016281 0.0016345 0.0020932 0.0020647 0.0019585 0.0019269] 0.0019444399513304234
[0.0016281  0.0016345  0.0020932  0.0020647  0.0019585  0.0019269
 0.00194444] 0.001906599965877831
[0.0016345  0.0020932  0.0020647  0.0019585  0.0019269  0.00194444
 0.0019066 ] 0.0018817000091075897
[0.0020932  0.0020647  0.0019585  0.0019269  0.00194444 0.0019066
 0.0018817 ] 0.00194240000564605
[0.0020647  0.0019585  0.0019269  0.00194444 0.0019066  0.0018817
 0.0019424 ] 0.00218260008841753
[0.0019585  0.0019269  0.00194444 0.

In [81]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
8/8 [==============================] - 1s 8ms/step - loss: 9.1984e-06
Epoch 2/200
8/8 [==============================] - 0s 7ms/step - loss: 3.2027e-06
Epoch 3/200
8/8 [==============================] - 0s 6ms/step - loss: 2.2035e-06
Epoch 4/200
8/8 [==============================] - 0s 6ms/step - loss: 2.1922e-06
Epoch 5/200
8/8 [==============================] - 0s 6ms/step - loss: 2.1836e-06
Epoch 6/200
8/8 [==============================] - 0s 6ms/step - loss: 2.1151e-06
Epoch 7/200
8/8 [==============================] - 0s 6ms/step - loss: 2.0363e-06
Epoch 8/200
8/8 [==============================] - 0s 6ms/step - loss: 1.9210e-06
Epoch 9/200
8/8 [==============================] - 0s 6ms/step - loss: 1.7550e-06
Epoch 10/200
8/8 [==============================] - 0s 6ms/step - loss: 1.8467e-06
Epoch 11/200
8/8 [==============================] - 0s 6ms/step - loss: 1.5647e-06
Epoch 12/200
8/8 [==============================] - 0s 6ms/step - loss: 1.5573e-06
Epoch 13/200


In [82]:
# demonstrate prediction
x_input = array([[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4],exchange[5],exchange[6]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


In [83]:
# onluk olarak yukarı yuvarlandı.
print("Next Month -> Prediction:",round(yhat[0][0]*10000,3), "$ Bitcoin/USD in", x.strftime('%B'))


Next Month -> Prediction: 16.439 $ Bitcoin/USD in January


## Year


In [84]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df)):
    if i % 30 == 0:  # it equals to weekly
        exchange_2.append(df['Şimdi'][i])


In [85]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[0.0016593 0.0017091 0.0020168 0.0019638 0.0019835] 0.0022611699532717466
[0.0017091  0.0020168  0.0019638  0.0019835  0.00226117] 0.0020169001072645187
[0.0020168  0.0019638  0.0019835  0.00226117 0.0020169 ] 0.0029923398979008198
[0.0019638  0.0019835  0.00226117 0.0020169  0.00299234] 0.0036039999686181545
[0.0019835  0.00226117 0.0020169  0.00299234 0.003604  ] 0.004319900181144476
[0.00226117 0.0020169  0.00299234 0.003604   0.0043199 ] 0.0038038999773561954
[0.0020169  0.00299234 0.003604   0.0043199  0.0038039 ] 0.004145400132983923
[0.00299234 0.003604   0.0043199  0.0038039  0.0041454 ] 0.004311100114136934
[0.003604  0.0043199 0.0038039 0.0041454 0.0043111] 0.005063400138169527
[0.0043199 0.0038039 0.0041454 0.0043111 0.0050634] 0.006325909867882729
[0.0038039  0.0041454  0.0043111  0.0050634  0.00632591] 0.005392500199377537
[0.0041454  0.0043111  0.0050634  0.00632591 0.0053925 ] 0.004608800169080496
[0.0043111  0.0050634  0.00632591 0.0053925  0.0046088 ] 0.004629299975931

In [86]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange_2[0], exchange_2[1], exchange_2[2], exchange_2[3], exchange_2[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
2/2 [==============================] - 1s 7ms/step - loss: 1.3195e-05
Epoch 2/300
2/2 [==============================] - 0s 6ms/step - loss: 6.8551e-06
Epoch 3/300
2/2 [==============================] - 0s 8ms/step - loss: 4.5451e-06
Epoch 4/300
2/2 [==============================] - 0s 7ms/step - loss: 6.5273e-06
Epoch 5/300
2/2 [==============================] - 0s 8ms/step - loss: 3.7811e-06
Epoch 6/300
2/2 [==============================] - 0s 7ms/step - loss: 4.9279e-06
Epoch 7/300
2/2 [==============================] - 0s 7ms/step - loss: 2.7237e-06
Epoch 8/300
2/2 [==============================] - 0s 7ms/step - loss: 4.1977e-06
Epoch 9/300
2/2 [==============================] - 0s 6ms/step - loss: 2.5674e-06
Epoch 10/300
2/2 [==============================] - 0s 6ms/step - loss: 3.3445e-06
Epoch 11/300
2/2 [==============================] - 0s 6ms/step - loss: 2.7841e-06
Epoch 12/300
2/2 [==============================] - 0s 6ms/step - loss: 3.1735e-06
Epoch 13/300


In [87]:
# onluk olarak yukarı yuvarlandı.
print(
    f"Next Year -> Prediction: {round(float(yhat[0][0]*10000),3)}  $ Bitcoin/USD in {x.strftime('%Y')} ")


Next Year -> Prediction: 17.277  $ Bitcoin/USD in 2023 
